In [5]:
#task 5 S11, onda sinusoidale con omega circa omega risonanza 4000hz

import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np

# -[Configurazione Analog Discovery 2]-----------------------------------------
#   1. Connessiene con AD2
ad2 = tdwf.AD2(iconfig=2)



ad2.vdd = +5
ad2.vss = -5
ad2.power(True)


#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)

index = 0
a = np.linspace(0.37, 0.57, 150)
offset = False
excitationFrequency = 4e3
offset = 0
#xy = True

allData = True

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=1e6 #3e6
scope.npt=16384
scope.ch1.rng = 5
scope.ch2.rng = 5
scope.trig(True, level = 1, sour = tdwf.trigsrcCh1)

# -[Funzioni di gestione eventi]-----------------------------------------------
def on_close(event):
    global flag_run
    flag_run = False
def on_key(event):
    global flag_run
    global flag_acq
    if event.key == 'x':  # => export su file
        filename = input("Esporta dati su file: ")
        #data = np.column_stack((scope.time.vals, scope.ch1.vals, scope.ch2.vals))

        

        #tt = np.array(flattened_tt)
        #ch1 = np.array(flattened_ch1)
        #ch2 = np.array(flattened_ch2)

#        print(str(type(tt)))
#        print(str(type(tt[1])))
#        print(len(tt))
#        print(tt)

        if(allData):
            flattened_tt = [item for sublist in tt for item in sublist]
            flattened_ch1 = [item for sublist in ch1 for item in sublist]
            flattened_ch2 = [item for sublist in ch2 for item in sublist]
#        print(str(type(flattened_tt)))
#        print(str(type(flattened_tt[1])))
#        print(len(flattened_tt))
#        print(flattened_tt)

            data = np.column_stack((flattened_tt, flattened_ch1, flattened_ch2))
        else:
            data = np.column_stack((tt, ch1, ch2))

        # Save the array to a file using np.savetxt
        np.savetxt("../Data/"+filename+'.txt', data, delimiter=' ')

        #np.savetxt("../Data/"+filename+".txt", np.c_(flattened_tt, flattened_ch1, flattened_ch2), delimiter = "\t")
        

        
    if event.key == ' ':  # => run/pausa misura
        flag_acq = not flag_acq
    if event.key == 'escape':  # => esci dalla misura
        flag_run = False

# -[Ciclo di misura]-----------------------------------------------------------
fig, axs = plt.subplots(1,2,figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect('key_press_event', on_key)
flag_run = True
flag_acq = True
flag_first = True

tt = []
ch1 = []
ch2 = []


wgen.w1.config(func = tdwf.funcSine, duty=50, freq = excitationFrequency, ampl = a[0])
if offset:
    wgen.w1.offs = offset
else:
    wgen.w1.offs = 0

wgen.w1.start()

while flag_run:
    if flag_acq: # l'acquisizione è attiva?
        scope.sample()
        tempTT = scope.time.vals
        tempCH1 = scope.ch1.vals
        tempCH2 = scope.ch2.vals

#        print('this is a ' + str(type(tempTT)))

        if(allData):
            tt.append(tempTT.copy().tolist())
            ch1.append(tempCH1.copy().tolist())
            ch2.append(tempCH2.copy().tolist())

            flattened_ch1 = [item for sublist in ch1 for item in sublist]
            flattened_ch2 = [item for sublist in ch2 for item in sublist]
        else:
            tt = tempTT.copy()
            ch1 = tempCH1.copy()
            ch2 = tempCH2.copy()

        
        



    # Visualizzazione
    if flag_first:
        flag_first = False
        hp1, = plt.plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        hp2, = plt.plot(scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
        plt.legend()
        plt.grid(True)
        plt.xlabel("Time [msec]", fontsize=15)
        plt.ylabel("Signal [V]", fontsize=15)
        plt.title("User interaction: x|space|escape")
        plt.tight_layout()
        plt.show(block = False)
        
        
#        wgen.w1.config(func = tdwf.funcDC)
#        wgen.w1.offs = -1.5
        
    else:

        if(flag_acq):
            index += 1
            if index >= a.size:
                index = a.size - 1
                flag_acq = False
            wgen.w1.ampl = a[index]
        

        axs[0].clear()
        axs[1].clear()

        
        #ax.plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        axs[1].plot(scope.ch1.vals, scope.ch2.vals, "-", label="Ch2(y)vsCh1(x)", color="tab:blue")
        axs[1].set_xlabel("Ch1 [V]", fontsize=15)
        axs[1].set_ylabel("Ch2 [V]", fontsize=15)

        #axs[0].plot(tt, ch1, "-", label="Ch1", color="tab:orange")
        #axs[0].plot(tt, ch2, "-", label="Ch2", color="tab:blue")
        axs[0].plot(tempTT, tempCH1, "-", label="Ch1", color="tab:orange")
        axs[0].plot(tempTT, tempCH2, "-", label="Ch2", color="tab:blue")
        axs[0].set_xlabel("Time [msec]", fontsize=15)
        axs[0].set_ylabel("Signal [V]", fontsize=15)
        #axs[0].plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        #axs[0].plot(scope.time.vals, scope.ch2.vals, "-", label="Ch2(y)", color="tab:blue")

        axs[0].set_title("User interaction: x|space|escape")
        axs[1].annotate(f"Ampl: {wgen.w1.ampl:.2f}", (0.5,0.5), xycoords='axes fraction', fontsize=15)
        axs[0].legend()
        axs[1].legend()
        axs[0].grid(True)
        axs[1].grid(True)

        fig.canvas.draw()
        fig.canvas.flush_events()

# -[Chiusura strumenti]--------------------------------------------------------
plt.close(fig)
ad2.close()

Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #2
Dispositivo disconnesso.


In [10]:
# Sinusoidal Excitation - frequency variation

import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np

# -[Configurazione Analog Discovery 2]-----------------------------------------
#   1. Connessiene con AD2
ad2 = tdwf.AD2(iconfig=2)



ad2.vdd = +5
ad2.vss = -5
ad2.power(True)


#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)

index = 0
f = np.linspace(3e3, 6.5e3, 300)
a = 0.458
offset = False
offsetVal = 0
#xy = True

allData = True

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=1e6 #3e6
scope.npt=16384
scope.ch1.rng = 5
scope.ch2.rng = 5
scope.trig(True, level = 1, sour = tdwf.trigsrcCh1)

# -[Funzioni di gestione eventi]-----------------------------------------------
def on_close(event):
    global flag_run
    flag_run = False
def on_key(event):
    global flag_run
    global flag_acq
    if event.key == 'x':  # => export su file
        filename = input("Esporta dati su file: ")
        #data = np.column_stack((scope.time.vals, scope.ch1.vals, scope.ch2.vals))

        

        #tt = np.array(flattened_tt)
        #ch1 = np.array(flattened_ch1)
        #ch2 = np.array(flattened_ch2)

#        print(str(type(tt)))
#        print(str(type(tt[1])))
#        print(len(tt))
#        print(tt)

        if(allData):
            flattened_tt = [item for sublist in tt for item in sublist]
            flattened_ch1 = [item for sublist in ch1 for item in sublist]
            flattened_ch2 = [item for sublist in ch2 for item in sublist]
#        print(str(type(flattened_tt)))
#        print(str(type(flattened_tt[1])))
#        print(len(flattened_tt))
#        print(flattened_tt)

            data = np.column_stack((flattened_tt, flattened_ch1, flattened_ch2))
        else:
            data = np.column_stack((tt, ch1, ch2))

        # Save the array to a file using np.savetxt
        np.savetxt("../Data/"+filename+'.txt', data, delimiter=' ')

        #np.savetxt("../Data/"+filename+".txt", np.c_(flattened_tt, flattened_ch1, flattened_ch2), delimiter = "\t")
        

        
    if event.key == ' ':  # => run/pausa misura
        flag_acq = not flag_acq
    if event.key == 'escape':  # => esci dalla misura
        flag_run = False

# -[Ciclo di misura]-----------------------------------------------------------
fig, axs = plt.subplots(1,2,figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect('key_press_event', on_key)
flag_run = True
flag_acq = True
flag_first = True

tt = []
ch1 = []
ch2 = []


wgen.w1.config(func = tdwf.funcSine, duty=50, freq = f[0], ampl = a)
if offset:
    wgen.w1.offs = offsetVal
else:
    wgen.w1.offs = 0

wgen.w1.start()

while flag_run:
    if flag_acq: # l'acquisizione è attiva?
        scope.sample()
        tempTT = scope.time.vals
        tempCH1 = scope.ch1.vals
        tempCH2 = scope.ch2.vals

#        print('this is a ' + str(type(tempTT)))

        if(allData):
            tt.append(tempTT.copy().tolist())
            ch1.append(tempCH1.copy().tolist())
            ch2.append(tempCH2.copy().tolist())

            flattened_ch1 = [item for sublist in ch1 for item in sublist]
            flattened_ch2 = [item for sublist in ch2 for item in sublist]
        else:
            tt = tempTT.copy()
            ch1 = tempCH1.copy()
            ch2 = tempCH2.copy()

        
        



    # Visualizzazione
    if flag_first:
        flag_first = False
        hp1, = plt.plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        hp2, = plt.plot(scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
        plt.legend()
        plt.grid(True)
        plt.xlabel("Time [msec]", fontsize=15)
        plt.ylabel("Signal [V]", fontsize=15)
        plt.title("User interaction: x|space|escape")
        plt.tight_layout()
        plt.show(block = False)
        
        
#        wgen.w1.config(func = tdwf.funcDC)
#        wgen.w1.offs = -1.5
        
    else:

        if(flag_acq):
            index += 1
            if index >= f.size:
                index = f.size - 1
                flag_acq = False
            wgen.w1.freq = f[index]
        

        axs[0].clear()
        axs[1].clear()

        
        #ax.plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        axs[1].plot(scope.ch1.vals, scope.ch2.vals, "-", label="Ch2(y)vsCh1(x)", color="tab:blue")
        axs[1].set_xlabel("Ch1 [V]", fontsize=15)
        axs[1].set_ylabel("Ch2 [V]", fontsize=15)

        #axs[0].plot(tt, ch1, "-", label="Ch1", color="tab:orange")
        #axs[0].plot(tt, ch2, "-", label="Ch2", color="tab:blue")
        axs[0].plot(tempTT, tempCH1, "-", label="Ch1", color="tab:orange")
        axs[0].plot(tempTT, tempCH2, "-", label="Ch2", color="tab:blue")
        axs[0].set_xlabel("Time [msec]", fontsize=15)
        axs[0].set_ylabel("Signal [V]", fontsize=15)
        #axs[0].plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        #axs[0].plot(scope.time.vals, scope.ch2.vals, "-", label="Ch2(y)", color="tab:blue")

        axs[0].set_title("User interaction: x|space|escape")
        axs[1].annotate(f"Ampl: {wgen.w1.freq:.2f}", (0.5,0.5), xycoords='axes fraction', fontsize=15)
        axs[0].legend()
        axs[1].legend()
        axs[0].grid(True)
        axs[1].grid(True)

        fig.canvas.draw()
        fig.canvas.flush_events()

# -[Chiusura strumenti]--------------------------------------------------------
plt.close(fig)
ad2.close()

Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #2
Dispositivo disconnesso.


In [37]:
ad2.close()

Dispositivo disconnesso.
